# **Import dependencies**

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import os
import glob
from pandas.core.groupby.groupby import isna

In [2]:
# Install owlready2
import sys
sys.path.insert(0, '/content/drive/MyDrive/Colab Notebooks/lib/Owlready2')
!pip install Owlready2 -q
from owlready2 import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 35.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# **Configuring paths and files**

In [ ]:
# Defining Paths
main_path = Path("/content/drive/MyDrive/Colab Notebooks/files/input")
year = '2021'
name_file_city = "Cities-IBGE.xls"
name_file_biome = "Cities_Biomes.xls"
name_file_climate = "Cities_climate_Koppen.xls"
name_file_code_mapbiomas = "Class_Code_Mapbiomas.xls"
name_file_taxas_emissao_cultivos = "Crop_Emission_Rate.xls"

In [ ]:
# Loading input datasets
cities_df = pd.read_excel (main_path / name_file_city,  sheet_name='DTB_2022_Municipio')
biomes_df = pd.read_excel (main_path / name_file_biome,  sheet_name='Planilha1')
climate_df = pd.read_excel (main_path / name_file_climate,  sheet_name='Data')
code_mapbiomas_df = pd.read_excel (main_path / name_file_code_mapbiomas,  sheet_name='Class_Code_Mapbiomas')
taxas_emissao_cultivos_df = pd.read_excel (main_path / name_file_taxas_emissao_cultivos, sheet_name='Emissao')

In [ ]:
# Resets dataframe indexes
taxas_emissao_cultivos_df.set_index(['Codigo Municipal IBGE'], drop=True, inplace=True)
cities_df.set_index(['Código Município Completo'], drop=True, inplace=True)
biomes_df.set_index(['CD_GEOCMU'], drop=True, inplace=True)
climate_df.set_index(['IBGE-Code'], drop=True, inplace=True)

In [ ]:
# Defining dictionaries
integrated_data_dic = {}
carbon_soil_ha_dic = {}
code_mapbiomas_dict = code_mapbiomas_df.to_dict('records')

In [ ]:
# Defining the list of areas where there are CO2 emission values (BRLUC)
list_of_accepted_areas = {1,3,4,5,9,10,12,13,14,15,18,19,20,21,36,39,40,41,46,47,48,49,62}

In [ ]:
# Accessing the desired year folder
current_path = main_path / year
current_path

PosixPath('/content/drive/MyDrive/Colab Notebooks/files/input/2021')

# **Generate files totaling by areas**

In [ ]:
biomes_dir = os.listdir(current_path)
biomes_dir = sorted(biomes_dir)
for biome_dir in biomes_dir:
    print('==================================================================================================================')
    print('Biome: ' + biome_dir)
    print('==================================================================================================================')
    current_path = current_path / biome_dir
    for root, subFolder, filename in os.walk(current_path):
      for folder in subFolder:
        city_cod = int(folder)
        internal_path = current_path / folder
        file_classes_area = internal_path.glob('*classes_area*.*')
        file_carbon_soil = internal_path.glob('*cos_0_30cm*.*')

        # Reset variables
        integrated_data_dic = {}

        # Recovering soil carbon values in each municipality
        for fileY in file_carbon_soil:
          # definindo o dataframe para carbono no solo
          carbon_soil_city_df = pd.read_csv(internal_path / fileY)
          for indice, linha in carbon_soil_city_df.iterrows():
            value_gt_cos = float(linha['Gt COS'])
            value_ton_cos = value_gt_cos * 1000000000
            value_ton_ha = value_ton_cos / float(linha['Área ha'])
            carbon_soil_ha_dic[linha['class']]= linha['class']
            carbon_soil_ha_dic[linha['class']] = value_ton_ha

          # Retrieving location data, carbon emissions and performing consolidations
          for fileX in file_classes_area:
            # Defining the dataframe for crop classes
            classe_area_df = pd.read_csv (internal_path / fileX)
            # Excluding columns 0
            classe_area_df.drop('0', axis=1, inplace=True)
            # Reaming the columns
            #classe_area_df = classe_area_df.rename(columns=lambda x: 'area_' + x)
            # Returning with the column name "Imovel"
            classe_area_df = classe_area_df.rename({"area_Imovel": "Imovel"}, axis='columns')
            # Replacing empty 'nan' values with 0
            classe_area_df.replace(np.nan, 0, inplace = True)
            print('---------------------')
            print('City code: ' + str(city_cod))
            print('---------------------')
            for indice, linha in classe_area_df.iterrows():
                for i in range(1,62):
                    if (linha.iloc[i]> 0.0):
                        if (int (classe_area_df.columns[i]) in list_of_accepted_areas):
                            farm = str(linha['Imovel'])
                            farm = farm[:-2]
                            key = str(linha['Imovel']) + '_' +  str(i)
                            area_cod = classe_area_df.columns[i]
                            area_name = code_mapbiomas_dict[int(classe_area_df.columns[i])-1]['Class']
                            area_size = linha.iloc[i]
                            CO_emission_ha = taxas_emissao_cultivos_df[int(classe_area_df.columns[i])][city_cod]
                            CO_emission_area = float(linha.iloc[i]) * (taxas_emissao_cultivos_df[int(classe_area_df.columns[i])][city_cod])
                            #print (area_name + ' - ' + classe_area_df.columns[i])
                            CO_stock_ha = carbon_soil_ha_dic[int(classe_area_df.columns[i])]
                            CO_stock_area = float(linha.iloc[i]) * float(carbon_soil_ha_dic[int(classe_area_df.columns[i])])
                            balance_CO2_area = float(CO_stock_area) - float(CO_emission_area)
                            balance_CO2_ha = float(balance_CO2_area) / float(area_size)

                            city_name = cities_df["Nome_Município"][city_cod]
                            city_name = city_name.replace(" ", "_")
                            state_name = cities_df["Nome_UF"][city_cod]
                            state_cod = str(city_cod)
                            state_cod = state_cod[:2]
                            biome = biomes_df["BIOMA"][city_cod]
                            match biome:
                                case ('Amazônia'):
                                    biome_cod = 1
                                case ('Caatinga'):
                                    biome_cod = 2
                                case ('Cerrado'):
                                    biome_cod = 3
                                case ('Mata Atlântica'):
                                    biome_cod = 4
                                case ('Pampa'):
                                    biome_cod = 5
                                case ('Pantanal'):
                                    biome_cod = 6
                            climate = climate_df["Köppen"][city_cod]
                            match climate:
                                case ('Af'):
                                    # hot climate, no dry season
                                    climate_cod = 1
                                case ('Am'):
                                    # hot monsoon climate
                                    climate_cod = 2
                                case ('As'):
                                    # warm weather with winter rain
                                    climate_cod = 3
                                case ('Aw'):
                                    # hot weather with summer rain
                                    climate_cod = 4
                                case ('BSh'):
                                    # semi-arid and hot
                                    climate_cod = 5
                                case ('Cfa'):
                                    # temperate climate, no dry season and hot summer
                                    climate_cod = 6
                                case ('Cfb'):
                                    # temperate climate, no dry season and cool summer
                                    climate_cod = 7
                                case ('Cwa'):
                                    # temperate climate with hot and humid summer
                                    climate_cod = 8
                                case ('Cwb'):
                                    # temperate climate with cool, humid summer
                                    climate_cod = 9

                            integrated_data_dic[key] = ({'farm': farm,'area_cod': area_cod, 'area_name': area_name, 'area_size': area_size,
                                                         'CO_emission_ha': CO_emission_ha, 'CO_emission_area': CO_emission_area, 'CO_stock_ha': CO_stock_ha,
                                                         'CO_stock_area': CO_stock_area, 'balance_CO2_ha': balance_CO2_ha, 'balance_CO2_area': balance_CO2_area,
                                                         'city_cod': city_cod, 'city_name': city_name, 'state_cod': state_cod, 'state_name': state_name,
                                                         'biome_cod': biome_cod,'biome_name': biome,'climate_cod': climate_cod,'climate_name': climate, 'year': year})

        # Creating a dataframe from the dictionary
        integrated_data_df = pd.DataFrame.from_dict(integrated_data_dic)

        # Transposing the dataframe to flip rows and columns
        integrated_data_df_transposed = integrated_data_df.T  # or df1.transpose()
        #print(integrated_data_df_transposed.head(1))

        # Changing the folder for saving files
        os.chdir(main_path)
        os.chdir(year)
        os.chdir(biome_dir)

        # Generating csv file with integrated data
        filename_out = 'data_integration_areas_' + str(city_cod) + '_' + city_name + '_' + year + '.csv'
        path_out = filename_out
        if os.path.exists(path_out):
          print ('Erro ao gerar o arquivo: ' + filename_out + '. Motivo: Arquivo existente!')
        else:
          integrated_data_df_transposed.to_csv(filename_out, index=True, header=True)
          print ('Arquivo: ' + filename_out + ' gerado com sucesso!')

      # Updating the path
      current_path = main_path / year


# **Generate files totaling by farm**

---



In [ ]:
areas_am_df = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/files/input/2021/data_integration_areas_areas-am-inference.csv')
areas_am_df

In [ ]:
#INCLUDING IN DATAFRAME

# Defining Paths
main_path = Path("/content/drive/MyDrive/Colab Notebooks/files/input")
year = '2021'

# Defining dataframes
areas_am_df = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/files/input/2021/data_integration_areas_areas-am-inference.csv')
columns_farms = ['index','farm','total_area_size', 'total_emission_area', 'total_stock_area','balance_CO2_farm', 'balance_CO2_farm_ha', 'city_cod','city_name',
           'state_cod','state_name','biome_cod','biome_name','climate_cod','climate_name','year']
farms_am_df = pd.DataFrame(columns=columns_farms)
farms_am_df

# Ordering the dataframe
areas_am_df = areas_am_df.sort_values(by='farm')
#areas_am_df.reset_index(drop=True, inplace=True)

# Defining variables
current_farm = areas_am_df.at[0, 'farm']
city_cod = areas_am_df.at[0, 'city_cod']
city_name = areas_am_df.at[0,'city_name']
#year = areas_am_df.at[0, 'year']
area_size_total = 0.0
CO2_emission_area_total = 0.0
CO2_stock_area_total = 0.0
balance_CO2_farm = 0.0
balance_CO2_farm_ha = 0.0
index_farm = 1
key = 1
count_elements = len(areas_am_df)

# Iterating over the lines and performing processing
for index, row in areas_am_df.iterrows():
  farm = row['farm']
  #print('passando no FOR-----------------------------------------------------------')
  #print('farm: ' + farm + '--- row[farm]: ' + (row['farm']))
  #print('current_farm: ' + current_farm)
  if (farm == current_farm):
    #print('---------------entrou IF')
    area_size_total = float(area_size_total) + float(row['area_size'])
    CO2_emission_area_total = float(CO2_emission_area_total) + float(row['CO2_emission_area'])
    CO2_stock_area_total = float(CO2_stock_area_total) + float(row['CO2_stock_area'])
    current_farm = farm
    #print('current_farm do IF: ' + current_farm + '\n')
    count_elements = count_elements -1
  else:
    #print('---------------entrou ELSE')
    balance_CO2_farm = CO2_stock_area_total - CO2_emission_area_total
    if (area_size_total != 0.0):
      balance_CO2_farm_ha = float(balance_CO2_farm) / float(area_size_total)
    farms_am_df.loc[key] = {'index': index_farm,'farm': current_farm.replace('farm_',''), 'total_area_size': area_size_total, 'total_emission_area': CO2_emission_area_total, 'total_stock_area': CO2_stock_area_total,
               'balance_CO2_farm': balance_CO2_farm, 'balance_CO2_farm_ha': balance_CO2_farm_ha, 'city_cod': areas_am_df.at[index, 'city_cod'],
               'city_name': areas_am_df.at[index,'city_name'],'state_cod': areas_am_df.at[index, 'state_cod'], 'state_name': areas_am_df.at[index,'state_name'],
               'biome_cod': areas_am_df.at[index, 'biome_cod'], 'biome_name': areas_am_df.at[index, 'biome_name'], 'climate_cod': areas_am_df.at[index, 'climate_cod'],
               'climate_name': areas_am_df.at[index, 'climate_name'], 'year': areas_am_df.at[index, 'year']}
    area_size_total = float(row['area_size'])
    CO2_emission_area_total = float(row['CO2_emission_area'])
    CO2_stock_area_total = float(row['CO2_stock_area'])
    current_farm = farm
    index_farm = index_farm + 1
    key = key + 1
    count_elements = count_elements -1
    if (count_elements == 0):
      balance_CO2_farm = CO2_stock_area_total - CO2_emission_area_total
      if (area_size_total != 0.0):
        balance_CO2_farm_ha = float(balance_CO2_farm) / float(area_size_total)
      farms_am_df.loc[key] = {'index': index_farm, 'farm': current_farm.replace('farm_',''), 'total_area_size': area_size_total, 'total_emission_area': CO2_emission_area_total, 'total_stock_area': CO2_stock_area_total,
                              'balance_CO2_farm': balance_CO2_farm, 'balance_CO2_farm_ha': balance_CO2_farm_ha, 'city_cod': areas_am_df.at[index, 'city_cod'],
                              'city_name': areas_am_df.at[index,'city_name'],'state_cod': areas_am_df.at[index, 'state_cod'], 'state_name': areas_am_df.at[index,'state_name'],
                              'biome_cod': areas_am_df.at[index, 'biome_cod'], 'biome_name': areas_am_df.at[index, 'biome_name'], 'climate_cod': areas_am_df.at[index, 'climate_cod'],
                              'climate_name': areas_am_df.at[index, 'climate_name'], 'year': areas_am_df.at[index, 'year']}
    #print('current_farm do ELSE: ' + current_farm + '--->key: ' + str(key) + '--->count_elements: ' + str(count_elements) +'\n')

# Changing the folder for saving files
os.chdir(main_path)
os.chdir(year)

# Generating csv file with integrated data
filename_out = 'data_integration_farms_' + str(city_cod) + '_' + city_name + '_' + year + '_final' + '.csv'
path_out = filename_out
if os.path.exists(path_out):
  print ('Erro ao gerar o arquivo: ' + filename_out + '. Motivo: Arquivo existente!')
else:
  farms_am_df.to_csv(filename_out, index=False, header=True)
  print ('Arquivo: ' + filename_out + ' gerado com sucesso!')

# Updating the path
current_path = main_path / year

Arquivo: data_integration_farms_1100908_Castanheiras_2021final.csv gerado com sucesso!


# **Preparing data for ontology**

In [ ]:
# Accessing the desired year folder
current_path = main_path / year
current_path

# Preparing datasets
biomes_dir = os.listdir(current_path)
biomes_dir = sorted(biomes_dir)
for biome_dir in biomes_dir:
    print('==================================================================================================================')
    print('Biome: ' + biome_dir)
    print('==================================================================================================================')
    current_path = current_path / biome_dir
    for root, subFolder, filename in os.walk(current_path):
      for folder in subFolder:
        city_cod = int(folder)
        internal_path = current_path / folder
        file_classes_area = internal_path.glob('*classes_area*.*')
        file_carbon_soil = internal_path.glob('*cos_0_30cm*.*')

        # Reset variables
        onto_data_dic = {}

        # Recovering soil carbon values in each municipality
        for fileY in file_carbon_soil:
          # definindo o dataframe para carbono no solo
          carbon_soil_city_df = pd.read_csv(internal_path / fileY)
          for indice, linha in carbon_soil_city_df.iterrows():
            value_gt_cos = float(linha['Gt COS'])
            value_ton_cos = value_gt_cos * 1000000000
            value_ton_ha = value_ton_cos / float(linha['Área ha'])
            carbon_soil_ha_dic[linha['class']]= linha['class']
            carbon_soil_ha_dic[linha['class']] = value_ton_ha

          # Retrieving location data, carbon emissions and performing consolidations
          for fileX in file_classes_area:
            # Defining the dataframe for crop classes
            classe_area_df = pd.read_csv (internal_path / fileX)
            # Excluding columns 0
            classe_area_df.drop('0', axis=1, inplace=True)
            # Reaming the columns
            #classe_area_df = classe_area_df.rename(columns=lambda x: 'area_' + x)
            # Returning with the column name "Imovel"
            classe_area_df = classe_area_df.rename({"area_Imovel": "Imovel"}, axis='columns')
            # Replacing empty 'nan' values with 0
            classe_area_df.replace(np.nan, 0, inplace = True)
            print('---------------------')
            print('City code: ' + str(city_cod))
            print('---------------------')
            for indice, linha in classe_area_df.iterrows():
                for i in range(1,62):
                    if (linha.iloc[i]> 0.0):
                        if (int (classe_area_df.columns[i]) in list_of_accepted_areas):
                            farm = str(linha['Imovel'])
                            farm = farm[:-2]
                            farm = 'farm_' + farm
                            key = str(linha['Imovel']) + '_' +  str(i)
                            area_cod = classe_area_df.columns[i]
                            area_cod = farm + '_area_' + area_cod
                            area_name = code_mapbiomas_dict[int(classe_area_df.columns[i])-1]['Class']
                            area_size = linha.iloc[i]
                            CO_emission_ha = taxas_emissao_cultivos_df[int(classe_area_df.columns[i])][city_cod]
                            #CO_emission_area = float(linha.iloc[i]) * (taxas_emissao_cultivos_df[int(classe_area_df.columns[i])][city_cod])
                            #print (area_name + ' - ' + classe_area_df.columns[i])
                            CO_stock_ha = carbon_soil_ha_dic[int(classe_area_df.columns[i])]
                            #CO_stock_area = float(linha.iloc[i]) * float(carbon_soil_ha_dic[int(classe_area_df.columns[i])])
                            #balance_CO2_area = float(CO_stock_area) - float(CO_emission_area)
                            #balance_CO2_ha = float(balance_CO2_area) / float(area_size)

                            city_name = cities_df["Nome_Município"][city_cod]
                            city_name = city_name.replace(" ", "_")
                            state_name = cities_df["Nome_UF"][city_cod]
                            state_name = state_name.replace(" ", "_")
                            state_cod = str(city_cod)
                            state_cod = state_cod[:2]
                            biome = biomes_df["BIOMA"][city_cod]
                            match biome:
                                case ('Amazônia'):
                                    biome_cod = 1
                                case ('Caatinga'):
                                    biome_cod = 2
                                case ('Cerrado'):
                                    biome_cod = 3
                                case ('Mata_Atlântica'):
                                    biome_cod = 4
                                case ('Pampa'):
                                    biome_cod = 5
                                case ('Pantanal'):
                                    biome_cod = 6
                            climate = climate_df["Köppen"][city_cod]
                            match climate:
                                case ('Af'):
                                    # hot climate, no dry season
                                    climate_cod = 1
                                case ('Am'):
                                    # hot monsoon climate
                                    climate_cod = 2
                                case ('As'):
                                    # warm weather with winter rain
                                    climate_cod = 3
                                case ('Aw'):
                                    # hot weather with summer rain
                                    climate_cod = 4
                                case ('BSh'):
                                    # semi-arid and hot
                                    climate_cod = 5
                                case ('Cfa'):
                                    # temperate climate, no dry season and hot summer
                                    climate_cod = 6
                                case ('Cfb'):
                                    # temperate climate, no dry season and cool summer
                                    climate_cod = 7
                                case ('Cwa'):
                                    # temperate climate with hot and humid summer
                                    climate_cod = 8
                                case ('Cwb'):
                                    # temperate climate with cool, humid summer
                                    climate_cod = 9

                            onto_data_dic[key] = ({'farm': farm,'area_cod': area_cod, 'area_name': area_name, 'area_size': area_size,
                                                    'CO2_emission_ha': CO_emission_ha, 'CO2_stock_ha': CO_stock_ha,'city_cod': city_cod,
                                                    'city_name': city_name, 'state_cod': state_cod, 'state_name': state_name,
                                                    'biome_cod': biome_cod,'biome_name': biome,'climate_cod': climate_cod,'climate_name': climate, 'year': year})

        # Creating a dataframe from the dictionary
        onto_data_df = pd.DataFrame.from_dict(onto_data_dic)

        # Transposing the dataframe to flip rows and columns
        onto_data_df_transposed = onto_data_df.T  # or df1.transpose()
        #print(integrated_data_df_transposed.head(1))

        # Changing the folder for saving files
        os.chdir(main_path)
        os.chdir(year)
        os.chdir(biome_dir)

        # Generating csv file with integrated data
        filename_out = 'data_integration_areas_' + str(city_cod) + '_' + city_name + '_' + year + '.csv'
        path_out = filename_out
        if os.path.exists(path_out):
          print ('Error generating the file: ' + filename_out + '. File already exists!')
        else:
          onto_data_df_transposed.to_csv(filename_out, index=True, header=True)
          print ('File: ' + filename_out + ' successfully generated!')

      # Updating the path
      current_path = main_path / year


**Consolidating datasets into one file**

In [ ]:
# Accessing the desired year folder
current_path = main_path / year
print(current_path)

# Consolidating datasets into one file
biomes_dir = os.listdir(current_path)
biomes_dir = sorted(biomes_dir)
print(biomes_dir)

for biome_dir in biomes_dir:
  print('==================================================================================================================')
  print('Biome: ' + biome_dir)
  print('==================================================================================================================')
  current_path = current_path / biome_dir
  print(current_path)
  os.chdir(current_path)

  extension = 'csv'
  #print (glob.glob('*.{}'.format(extension)))
  all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

  # Concatenating all files in the folder
  combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames])

  # Generating a dataframe to create an autonumbered index
  all_areas_df = combined_csv
  all_areas_df = all_areas_df.drop(columns=['Unnamed: 0'])
  all_areas_df.insert(0, 'index', range(1, 1 + len(all_areas_df)))


  # Generating the biomes output file
  current_path = main_path / year
  os.chdir(current_path)
  filename_out = 'data_integration_all_areas_' + str(biome_dir) + '.csv'
  path_out = filename_out
  if os.path.exists(path_out):
    print ('Error generating the file: ' + filename_out + '. File already exists!')
  else:
    all_areas_df.to_csv(filename_out, index=False, encoding='utf-8-sig')
    print ('Output file generated successfully:  ')

# Generating the final output file
current_path = main_path / year
print(current_path)
os.chdir(current_path)

extension = 'csv'
all_filenames_final = [i for i in glob.glob('*.{}'.format(extension))]

# Concatenating all files in the folder
combined_csv_final = pd.concat([pd.read_csv(f) for f in all_filenames_final])

# Generating a dataframe to create an autonumbered index
all_areas_final_df = combined_csv_final
all_areas_final_df = all_areas_final_df.drop(columns=['index'])
all_areas_final_df.insert(0, 'index', range(1, 1 + len(all_areas_final_df)))

filename_out = 'data_integration_all_areas_final.csv'
path_out = filename_out
if os.path.exists(path_out):
  print ('Error generating the file: ' + filename_out + '. File already exists!')
else:
  all_areas_final_df.to_csv(filename_out, index=False, encoding='utf-8-sig')
  print ('Output file generated successfully:  ')


# **Create ontology**

In [ ]:
from owlready2.triplelite import Graph
# Insert a directory to store ontologies

onto_path.append("/content/drive/MyDrive/Colab Notebooks/files/onto")
default_world.set_backend(filename = "/content/drive/MyDrive/Colab Notebooks/files/onto/quadstore-areas-am.sqlite3")
onto = get_ontology("/content/drive/MyDrive/Colab Notebooks/files/onto/CarbonOnto4-0.owl").load(reload_if_newer = True)
default_world.save()

#open bd
#graph = Graph("/content/drive/MyDrive/Colab Notebooks/files/onto/quadstore.sqlite3")
#default_world.set_backend(graph)

# **Populating the ontology**

In [ ]:
f = open("/content/drive/MyDrive/Colab Notebooks/files/input/2021/data_integration_all_areas_amazonia.csv")
reader = csv.reader(f)
next(reader)

with onto:
  for row in reader:
    id, farm, area_cod, area_name, area_size, CO_emission_ha, CO_stock_ha, city_cod, city_name, state_cod, state_name, biome_cod, biome_name, climate_cod, climate_name, year = row

    individual = onto.Farm(farm)

    individual2 = onto.City(city_name)
    individual.isPartOf.append(individual2)

    individual3 = onto.State(state_name)
    individual2.isPartOf.append(individual3)

    individual4 = onto.Biome(biome_name)
    individual2.isPartOf.append(individual4)

    individual5 = onto.Climate(climate_name)
    individual.hasClimate.append(individual5)

    individual6 = onto.Farm_Area(area_cod)
    individual.hasPart.append(individual6)
    individual6.hasSizeFarmArea.append(float(area_size))
    individual6.hasEmissionValueCO2FarmAreaHa.append(float(CO_emission_ha))
    individual6.hasSequestrationValueCO2FarmAreaHa.append(float(CO_stock_ha))
    individual6.hasClimate.append(individual5)

onto.save('/content/drive/MyDrive/Colab Notebooks/files/onto/CarbonOnto4-0-individuals-areas-am.owl')

In [ ]:
default_world.set_backend(filename = "/content/drive/MyDrive/Colab Notebooks/files/onto/quadstore-inferences-areas-am.sqlite3")
onto_inferences = get_ontology("/content/drive/MyDrive/Colab Notebooks/files/onto/CarbonOnto4-0-inferences-areas-am.owl")
with onto_inferences:
  sync_reasoner_pellet(infer_property_values = True, infer_data_property_values = True)
onto_inferences.save("/content/drive/MyDrive/Colab Notebooks/files/onto/CarbonOnto4-0-inferences-areas-am.owl")
default_world.save()

In [ ]:
# Retrieving records after inference in the database
records = default_world.graph.execute('SELECT iri FROM resources WHERE (length(iri) > 150) AND  (iri like "%hasEmissionValueCO2FarmArea%" OR iri like "%hasSequestrationValueCO2FarmArea%" OR iri like "%hasCO2BalanceFarmArea%" OR iri like "%hasCO2BalanceFarmAreaHa%")').fetchall()
records_df = pd.DataFrame(records)
records_df

In [ ]:
# Retrieving records after inference in the database
records_areas_am = default_world.graph.execute('SELECT iri FROM resources WHERE (length(iri) > 150) AND  (iri like "%hasEmissionValueCO2FarmArea%" OR iri like "%hasSequestrationValueCO2FarmArea%" OR iri like "%hasCO2BalanceFarmArea%" OR iri like "%hasCO2BalanceFarmAreaHa%")').fetchall()
records_df_am = pd.DataFrame(records_areas_am)
records_df_am

current_path = ("/content/drive/MyDrive/Colab Notebooks/files/output")
print(current_path)
os.chdir(current_path)
filename_out = 'records_areas_am_onto.csv'
path_out = filename_out
print(os.getcwd() + '/' + filename_out)
if os.path.exists(path_out):
  print ('Error generating the file: ' + filename_out + '. File already exists!')
else:
  records_df_am.to_csv(filename_out, index=False, encoding='utf-8-sig')
  print ('Output file generated successfully:  ')

# **Generating scripts for ML**

In [ ]:
# Defining dataframe
areas_am_df = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/files/input/2021/data_integration_all_areas_amazonia.csv')
areas_am_df.insert(6, "CO2_emission_area","")
areas_am_df.insert(8, "CO2_stock_area","")
areas_am_df.insert(9, "balance_CO2_ha","")
areas_am_df.insert(10, "balance_CO2_area","")

# Loading inferecing records
records_inference_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/files/onto/records_areas_am_onto.csv')
for index, row in records_inference_df.iterrows():
  string = row[0]
  string = string.replace(" ", "\n")
  string = string.replace(",()", "\n")
  string = string.replace("literal(", "")
  string = string.replace("#", "\n")
  string_list = string.split()
  del string_list[0:2]
  del string_list[1]
  del string_list[3:5]
  farm_area_cod = string_list[0]
  reqd_Index = areas_am_df[areas_am_df['area_cod'] == farm_area_cod].index.tolist()
  match string_list[1]:
    case ('hasEmissionValueCO2FarmArea'):
      areas_am_df.at[reqd_Index[0], 'CO2_emission_area'] = string_list[2]
      #print(areas_am_df.iloc[reqd_Index[0]])
    case ('hasSequestrationValueCO2FarmArea'):
      areas_am_df.at[reqd_Index[0], 'CO2_stock_area'] = string_list[2]
      areas_am_df.iloc[reqd_Index[0]]
    case ('hasCO2BalanceFarmAreaHa'):
      areas_am_df.at[reqd_Index[0], 'balance_CO2_ha'] = string_list[2]
      areas_am_df.iloc[reqd_Index[0]]
    case ('hasCO2BalanceFarmArea'):
      areas_am_df.at[reqd_Index[0], 'balance_CO2_area'] = string_list[2]
      areas_am_df.iloc[reqd_Index[0]]
#areas_am_df

# Standardizing data labels
for index, row in areas_am_df.iterrows():
  areas_am_df.at[index, 'farm'] = row['farm'].replace('farm_','')
  areas_am_df.at[index, 'area_cod'] = row['area_cod'][-2:].replace('_','')
areas_am_df

# Generating csv file with integrated data
current_path = ('/content/drive/MyDrive/Colab Notebooks/files/input/2021')
print(current_path)
os.chdir(current_path)
filename_out = 'data_integration_areas-am-inference-teste.csv'
path_out = filename_out
if os.path.exists(path_out):
  print ('Error generating the file: ' + filename_out + '. File already exists!')
else:
  areas_am_df.to_csv(filename_out, index=False, header=True)
  print ('File: ' + filename_out + ' successfully generated!')

/content/drive/MyDrive/Colab Notebooks/files/input/2021
File: data_integration_areas-am-inference-teste.csv successfully generated!


In [ ]:
  # PARA TESTES
  string = 'DATAPROPVAL: http://www.semanticweb.org/administrador/ontologies/2023/05/CarbonOnto4-0#farm_1001178_area_15 http://www.semanticweb.org/administrador/ontologies/2023/05/CarbonOnto4-0#hasEmissionValueCO2FarmArea literal(18.0496,(),http://www.w3.org/2001/XMLSchema#decimal)'
  #print(string)
  string = string.replace(" ", "\n")
  string = string.replace(",()", "\n")
  string = string.replace("literal(", "")
  string = string.replace("#", "\n")
  string_list = string.split()
  #print(string_list)
  del string_list[0:2]
  del string_list[1]
  del string_list[3:5]
  print(string_list)
  farm = string_list[0]
  print(farm)


In [ ]:
# PARA TESTES
areas_am_df = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/files/input/2021/data_integration_all_areas_amazonia.csv')
areas_am_df.insert(6, "CO2_emission_area","")
areas_am_df.insert(8, "CO2_stock_area","")
areas_am_df.insert(9, "balance_CO2_ha","")
areas_am_df.insert(10, "balance_CO2_area","")
reqd_Index = areas_am_df[areas_am_df['area_cod'] == 'farm_1001178_area_15'].index.tolist()
print(reqd_Index[0])
areas_am_df.at[reqd_Index[0], 'balance_CO2_area'] = 95.8989953465111635
areas_am_df.iloc[reqd_Index[0]]
areas_am_df